# Playing recursively with trees

This is not a book about programming in Python, as the topic is vast. Having said that, it’s not
common for introductory Python books to discuss **recursive programming** at length. Usually, recursive
programming techniques are well tailored to **deal with trees**. It is also a required programming strategy
with functional programming dialects, which can be quite useful when you perform concurrent
processing. This is common when processing very large datasets.

The phylogenetic notion of a tree is slightly different from that in computer science. Phylogenetic trees
can be **rooted** (if so, then they are normal tree data structures) or **unrooted**, making them undirected
acyclic graphs. Additionally, phylogenetic trees can have **weights on their edges**. Therefore, be mindful
of this when you read the documentation; if the text is written by a phylogeneticist, you can expect
the tree (rooted and unrooted), while most other documents will use undirected acyclic graphs for
unrooted trees. In this recipe, we will assume that all of the trees are rooted.

Finally, note that while this recipe is mostly devised to help you understand recursive algorithms and
tree-like structures, the final part is actually quite practical and fundamental for the next recipe to work.

In [1]:
import dendropy

ebola_raxml = dendropy.Tree.get_from_path('my_ebola.nex', 'nexus')

## Compute the level of each node:

In [2]:
def compute_level(node, level=0):
    for child in node.child_nodes():
        compute_level(child, level + 1)
    if node.taxon is not None:
        print("%s: %d %d" % (node.taxon, node.level(), level))

compute_level(ebola_raxml.seed_node)

'BDBV_KC545396 18728 bp': 2 2
'BDBV_FJ217161 18728 bp': 3 3
'TAFV_FJ217162 18728 bp': 4 4
'EBOV_2014_KM034561 18728 bp': 11 11
'EBOV_2014_KM034554 18728 bp': 11 11
'EBOV_2014_KM034555 18728 bp': 10 10
'EBOV_2014_KM034562 18728 bp': 10 10
'EBOV_2014_KM034558 18728 bp': 12 12
'EBOV_2014_KM034551 18728 bp': 12 12
'EBOV_2014_KM034557 18728 bp': 13 13
'EBOV_2014_KM034556 18728 bp': 14 14
'EBOV_2014_KM034560 18728 bp': 14 14
'EBOV_2014_KM034553 18728 bp': 13 13
'EBOV_2014_KM034552 18728 bp': 13 13
'EBOV_2014_KM034550 18728 bp': 9 9
'EBOV_2014_KM034563 18728 bp': 10 10
'EBOV_2014_KM034549 18728 bp': 10 10
'EBOV_2014_KM034559 18728 bp': 7 7
'EBOV_1995_KC242796 18728 bp': 9 9
'EBOV_1995_KC242799 18728 bp': 9 9
'EBOV_1976_KC242801 18728 bp': 9 9
'EBOV_1976_AF272001 18728 bp': 9 9
'EBOV_2007_KC242790 18728 bp': 9 9
'EBOV_2007_KC242785 18728 bp': 9 9
'EBOV_2007_KC242784 18728 bp': 9 9
'EBOV_2007_KC242786 18728 bp': 11 11
'EBOV_2007_KC242789 18728 bp': 11 11
'EBOV_2007_KC242788 18728 bp': 11 11
'EB

DendroPy’s node representation has a level method (which is used for comparison), but the
point here is to introduce a recursive algorithm, so we will implement it anyway.

Note how the function works; it’s **called with seed_node** (which is the root node, since the
code works under the assumption that we are dealing with rooted trees). The default level for
the **root node is 0**. The function will then **call itself** for all its children nodes, increasing the
level by one. Then, for each node that is not a leaf (that is, it is internal to the tree), the calling
will be repeated, and this will recurse until we get to the leaf nodes.

For the leaf nodes, we then print the level (we could have done the same for the internal nodes)
and show the same information computed by DendroPy’s internal function.

## Compute the height of each node:

In [3]:
def compute_height(node):
    children = node.child_nodes()
    if len(children) == 0:
        height = 0
    else:
        height = 1 + max(map(lambda x: compute_height(x), children))
    desc = node.taxon or 'Internal'
    print("%s: %d %d" % (desc, height, node.level()))
    return height

compute_height(ebola_raxml.seed_node)

'BDBV_KC545396 18728 bp': 0 2
'BDBV_FJ217161 18728 bp': 0 3
'TAFV_FJ217162 18728 bp': 0 4
'EBOV_2014_KM034561 18728 bp': 0 11
'EBOV_2014_KM034554 18728 bp': 0 11
Internal: 1 10
'EBOV_2014_KM034555 18728 bp': 0 10
Internal: 2 9
'EBOV_2014_KM034562 18728 bp': 0 10
'EBOV_2014_KM034558 18728 bp': 0 12
'EBOV_2014_KM034551 18728 bp': 0 12
Internal: 1 11
'EBOV_2014_KM034557 18728 bp': 0 13
'EBOV_2014_KM034556 18728 bp': 0 14
'EBOV_2014_KM034560 18728 bp': 0 14
Internal: 1 13
Internal: 2 12
'EBOV_2014_KM034553 18728 bp': 0 13
'EBOV_2014_KM034552 18728 bp': 0 13
Internal: 1 12
Internal: 3 11
Internal: 4 10
Internal: 5 9
Internal: 6 8
'EBOV_2014_KM034550 18728 bp': 0 9
'EBOV_2014_KM034563 18728 bp': 0 10
'EBOV_2014_KM034549 18728 bp': 0 10
Internal: 1 9
Internal: 2 8
Internal: 7 7
'EBOV_2014_KM034559 18728 bp': 0 7
Internal: 8 6
'EBOV_1995_KC242796 18728 bp': 0 9
'EBOV_1995_KC242799 18728 bp': 0 9
Internal: 1 8
'EBOV_1976_KC242801 18728 bp': 0 9
'EBOV_1976_AF272001 18728 bp': 0 9
Internal: 1 8
I

14

Here, we will use the same recursive strategy, but each node will return its height to its parent.
If the node is a leaf, then the height is 0; if not, then it’s 1 plus the maximum height of its
entire offspring.

Note that we use a map over a lambda function to get the heights of all the children of the current
node. Then, we choose the maximum (the max function performs a reduce operation here
because it summarizes all of the values that are reported). If you are relating this to MapReduce
frameworks, you are correct; they are inspired by functional programming dialects like these.

## Compute the number of offspring for each node:

In [13]:
def compute_nofs(node):
    children = node.child_nodes()
    nofs= len(children)
    map(lambda x: compute_nofs(x), children)
    desc = node.taxon or 'Internal'
    print("%s: %d %d" % (desc, nofs, node.level()))

compute_nofs(ebola_raxml.seed_node)

Internal: 3 0


## print all of the leaves

In [6]:
def print_nodes(node):
    for child in node.child_nodes():
        print_nodes(child)
    if node.taxon is not None:
        print('%s (%d)' % (node.taxon, node.level()))

print_nodes(ebola_raxml.seed_node)

'BDBV_KC545396 18728 bp' (2)
'BDBV_FJ217161 18728 bp' (3)
'TAFV_FJ217162 18728 bp' (4)
'EBOV_2014_KM034561 18728 bp' (11)
'EBOV_2014_KM034554 18728 bp' (11)
'EBOV_2014_KM034555 18728 bp' (10)
'EBOV_2014_KM034562 18728 bp' (10)
'EBOV_2014_KM034558 18728 bp' (12)
'EBOV_2014_KM034551 18728 bp' (12)
'EBOV_2014_KM034557 18728 bp' (13)
'EBOV_2014_KM034556 18728 bp' (14)
'EBOV_2014_KM034560 18728 bp' (14)
'EBOV_2014_KM034553 18728 bp' (13)
'EBOV_2014_KM034552 18728 bp' (13)
'EBOV_2014_KM034550 18728 bp' (9)
'EBOV_2014_KM034563 18728 bp' (10)
'EBOV_2014_KM034549 18728 bp' (10)
'EBOV_2014_KM034559 18728 bp' (7)
'EBOV_1995_KC242796 18728 bp' (9)
'EBOV_1995_KC242799 18728 bp' (9)
'EBOV_1976_KC242801 18728 bp' (9)
'EBOV_1976_AF272001 18728 bp' (9)
'EBOV_2007_KC242790 18728 bp' (9)
'EBOV_2007_KC242785 18728 bp' (9)
'EBOV_2007_KC242784 18728 bp' (9)
'EBOV_2007_KC242786 18728 bp' (11)
'EBOV_2007_KC242789 18728 bp' (11)
'EBOV_2007_KC242788 18728 bp' (11)
'EBOV_2007_KC242787 18728 bp' (11)
'RESTV_FJ621

Note that all the functions that we have developed so far impose a very clear traversal pattern
on the tree. It calls its first offspring, then that offspring will call their offspring, and so on; only
after this will the function be able to call its next offspring in a **depth-first** pattern. However,
we can do things differently.

In [14]:
from collections import deque

def print_breadth(tree):
    queue = deque()
    queue.append(tree.seed_node)
    while len(queue) > 0:
        process_node = queue.popleft()
        if process_node.taxon is not None:
            print('%s (%d)' % (process_node.taxon, process_node.level()))
        else:
            for child in process_node.child_nodes():
                queue.append(child)

print_breadth(ebola_raxml)

'BDBV_KC545393 18728 bp' (1)
'BDBV_KC545396 18728 bp' (2)
'BDBV_KC545394 18728 bp' (2)
'BDBV_KC545395 18728 bp' (2)
'BDBV_FJ217161 18728 bp' (3)
'TAFV_FJ217162 18728 bp' (4)
'EBOV_2014_KM034559 18728 bp' (7)
'RESTV_FJ621584 18728 bp' (7)
'SUDV_KC589025 18728 bp' (8)
'SUDV_EU338380 18728 bp' (8)
'EBOV_2014_KM034550 18728 bp' (9)
'EBOV_1995_KC242796 18728 bp' (9)
'EBOV_1995_KC242799 18728 bp' (9)
'EBOV_1976_KC242801 18728 bp' (9)
'EBOV_1976_AF272001 18728 bp' (9)
'EBOV_2007_KC242790 18728 bp' (9)
'EBOV_2007_KC242785 18728 bp' (9)
'EBOV_2007_KC242784 18728 bp' (9)
'RESTV_JX477165 18728 bp' (9)
'RESTV_FJ621583 18728 bp' (9)
'RESTV_FJ621585 18728 bp' (9)
'SUDV_JN638998 18728 bp' (9)
'SUDV_AY729654 18728 bp' (9)
'SUDV_FJ968794 18728 bp' (9)
'SUDV_KC242783 18728 bp' (9)
'EBOV_2014_KM034555 18728 bp' (10)
'EBOV_2014_KM034562 18728 bp' (10)
'EBOV_2014_KM034563 18728 bp' (10)
'EBOV_2014_KM034549 18728 bp' (10)
'RESTV_AB050936 18728 bp' (10)
'RESTV_JX477166 18728 bp' (10)
'EBOV_2014_KM034561 1872

Let’s get back to the real dataset. As we have a bit too much data to visualize, we will generate
a trimmed-down version, where we remove the subtrees that have single species (in the case
of EBOV, they have the same outbreak). We will also ladderize the tree, that is, sort the child
nodes in order of the number of children

In [15]:
from copy import deepcopy
simple_ebola = deepcopy(ebola_raxml)

def simplify_tree(node):
    prefs = set()
    for leaf in node.leaf_nodes():
        my_toks = leaf.taxon.label.split(' ')[0].split('_')
        if my_toks[0] == 'EBOV':
            prefs.add('EBOV' + my_toks[1])
        else:
            prefs.add(my_toks[0])
    if len(prefs) == 1:
        print(prefs, len(node.leaf_nodes()))
        node.taxon = dendropy.Taxon(label=list(prefs)[0])
        #node.collapse_clade()
        node.set_child_nodes([])
    else:
        for child in node.child_nodes():
            simplify_tree(child)

simplify_tree(simple_ebola.seed_node)
simple_ebola.ladderize()
simple_ebola.write_to_path('ebola_simple.nex', 'nexus')

{'BDBV'} 1
{'BDBV'} 1
{'TAFV'} 1
{'EBOV2014'} 15
{'EBOV1995'} 2
{'EBOV1976'} 2
{'EBOV2007'} 7
{'RESTV'} 6
{'SUDV'} 6
{'BDBV'} 2
{'BDBV'} 1


We will perform a **deep copy** of the tree structure. As our function and the **ladderization** are **destructive**
(they will change the tree), we will want to maintain the original tree.

DendroPy is able to enumerate all the leaf nodes (at this stage, a good exercise would be to write a
function to perform this). With this functionality, we will get all the leaves for a certain node. If they
share the same species and outbreak year as in the case of EBOV, we remove all of the child nodes,
leaves, and internal subtree nodes.